In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import sklearn.metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss, jaccard_score
from sklearn.metrics import f1_score

In [2]:
from data_quality import cargue_datasets_c
casos, hurto_tp, mede_victimas, compar = cargue_datasets_c()
hurto_ml = hurto_tp 

from data_quality import casos_c
casos=casos_c(casos)

from data_quality import hurto_tp_c
hurto_tp= hurto_tp_c(hurto_tp)

from data_quality import medevic_c
mede_victimas = medevic_c(mede_victimas)

In [3]:
# Codificar variables categóricas (como Sexo, Estado_civil, Transporte, Modalidad, Conducta, Comuna, Lugar, Bien)
categorical_cols = ['Sexo', 'Estado_civil', 'Transporte', 'Modalidad', 'Conducta', 'Comuna', 'Lugar', 'Bien']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    hurto_ml[col] = le.fit_transform(hurto_ml[col])
    label_encoders[col] = le

In [4]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = hurto_ml.drop(['Fecha','Sexo'], axis=1)  # Elimina las columnas
y = hurto_ml['Sexo']  # Define tu objetivo, en este caso 'Sexo'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
def compare_metrics(learners,X,y,metrics,train_size,repeats):
    strlearners=[]
    for i in range(0,len(learners)):
        strlearners.append(str(learners[i]))  
    df=pd.DataFrame(columns=metrics,index=strlearners)
    for i in range (0,len(learners)):
        for j in range(0,len(metrics)):
            scores=[]
            for k in range(repeats):
                X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=train_size)
                learners[i].fit(X_train,y_train)
                y_hat = learners[i].predict(X_test)
                y_prob = learners[i].predict_proba(X_test)
                if metrics[j]=="Log loss":
                    scores.append(sklearn.metrics.log_loss(y_test, y_prob))
                if metrics[j]=="Brier Score Loss":
                    scores.append(sklearn.metrics.brier_score_loss(y_test, y_prob[:,1]))
                if metrics[j]=="Accuracy":
                    scores.append(sklearn.metrics.accuracy_score(y_test, y_hat))
                if metrics[j]=="Precision":
                    scores.append(sklearn.metrics.precision_score(y_test, y_hat))
                if metrics[j]=="F1":
                    scores.append(sklearn.metrics.f1_score(y_test, y_hat))
                if metrics[j]=="Recall":
                    scores.append(sklearn.metrics.recall_score(y_test, y_hat))
                if metrics[j]=="Auroc":
                    scores.append(sklearn.metrics.roc_auc_score(y_test, y_prob[:,1]))
            df.iloc[i,j]=np.mean(scores)
    return df

In [6]:
metricas=["Log loss","Brier Score Loss","Accuracy","Precision","F1","Recall","Auroc"]
decision_tree=DecisionTreeClassifier()
naive_bayes=GaussianNB()
kNN=KNeighborsClassifier()
log_regression = LogisticRegression()
RandomForest = RandomForestClassifier()
elearn=[decision_tree,naive_bayes,kNN,log_regression,RandomForest]

In [7]:
#hurto_ml_scores  = compare_metrics(elearn,X,y,metricas,0.8,20)
#hurto_ml_scores

##### Multietiqueta

In [23]:
bins = [-1, 12, 18, 30, 60, float('inf')]
labels = ['Niño', 'Adolescente', 'Adulto joven', 'Adulto', 'Adulto mayor']

# Utiliza pd.cut para crear los intervalos y asignar las etiquetas
hurto_ml['grupo_edad'] = pd.cut(hurto_ml['Edad'], bins=bins, labels=labels, right=False)

In [20]:
categorical_cols = ['grupo_edad']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    hurto_ml[col] = le.fit_transform(hurto_ml[col])
    label_encoders[col] = le

In [10]:
# Dividir el conjunto de datos en características (X) y etiquetas (y)
X = hurto_ml.drop(['Fecha', 'grupo_edad', 'Sexo','Edad'], axis=1)
y = list(zip(hurto_ml['grupo_edad'],hurto_ml['Sexo']))

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un clasificador multi-etiqueta (por ejemplo, Random Forest)
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

# Entrenar el modelo
classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = classifier.predict(X_test)

# Evaluar el rendimiento del modelo
print(classifier.score(X_train, np.array(y_train)))

0.804208065458796


In [52]:
X_train

,Estado_civil,Transporte,Modalidad,Conducta,Comuna,Lugar,Bien,Año
1144,3,0,3,11,4,66,43,2017
19335,3,1,3,11,9,24,43,2021
19629,3,2,0,11,9,4,43,2021
4630,3,0,0,8,9,69,128,2014
10628,3,0,3,11,15,44,43,2019
...,...,...,...,...,...,...,...,...
21575,0,1,3,11,9,27,43,2022
5390,4,2,0,4,4,69,128,2015
860,3,0,3,11,13,44,43,2017
15795,3,1,3,11,9,27,53,2018


In [54]:
B_grupo_edad = {1:'Adulto', 2:'Adulto joven',3:'Adulto mayor',0:'Adolescente',4:'Niño'}
B_genero = {0:'Hombre', 1:'Mujer'}

In [58]:
test = [(1,2,3,11,13,66,43,2033)]
y_pred = classifier.predict(test)

print('Género predicción:', B_genero[y_pred[0, 1]],'\n','Grupo edad predicción:', B_grupo_edad[y_pred[0, 0]])

Género predicción: Hombre 
 Grupo edad predicción: Adulto


c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
